# Project report - Continous Control

In this project the goal is to train a robotic arm to reach a  goal in space. Most work on reinforced learning agents are on discrete action spaces to e.g. learn how to play Atari games from pixels (like DQN - Deep Q-learning). The problem with robotic control is that the action space is continous - which is a more challenging task than discrete actions due to a possible infinite action space. 


## The DDPG algorithm

A algorithm called DDPG is developed to learn problems that have continous action spaces (here: controlling the motors and forces of the robot) and can be seen as DQN with continious actions. The robot model returns a state vector with all the info about the state of the robotic arm (not pixels) of 33 real values. The action vector (4 real values) is applied the model which replies with reward value and a new state vector. This is what we need to create a reinforced training agent that can learn the behaviour of the robotic arm without a model (model-free).

DDPG is an model-free actor-critic method. The actor estimates the best actions to take and the critic estimates the $Q(s, a)$ value for the state/action pairs. The $Q(s, a)$ is learned using the Bellman equation as in Q-learning. 


$Q^{\pi}(s,a) = \sum_{s'} \mathcal{P}_{s s'}^{a} \Bigg[ \mathcal{R}_{s s'}^{a} + \gamma \sum_{a'} \mathbb{E}_{\pi} \Big[ \sum_{k=0}^{\infty} \gamma^k r_{t+k+2} | s_{t+1} = s', a_{t+1} = a' \Big] \Bigg]$

$Q^{\pi}(s,a) = \sum_{s'} \mathcal{P}_{s s'}^{a} \Big[ \mathcal{R}_{s s'}^{a} + \gamma \sum_{a'} \pi (s', a') Q^{\pi}(s', a') \Big]$

The actor network output the believed best action in all states. This is a deterministic policy and the actor is learning $argmax_{a} Q(s,a)$ The value function is estimated like this and is an approximate maximizer:

$\mu(s; \theta_{\mu})$

The critic learn to evaluate the optimal value action using the actors suggested action.

$Q(s, \mu (s; \theta_{\mu}); \theta_{Q}) $

In order to make these network learn stable, we need to use two tricks:

 ### Use of replay buffer
 In order to decorreleate the learning examples a replay buffer is used in the same way as in DQN.
 
 ### Use of soft update
 
In DDPG there two pairs of neural networks. One (local and target) for the actor and one (local and target) for the critic. The updates are soft updates which makes sure the weights change slowly blends into the target network. This is different than in  DQN where the weights are copied (raw) to the target network at regular intervals.
   
### Exploration of action space
In a continious action space, as solved by DDPG, the exploration is added by adding random noise to the action values that are selected by the network. This is very different that the $\epsilon$ -greedy approach used in e.g. DQN. We are using the Ornstein-Uhlenbeck distribution for this exploration.




![DDPG](img/ddpg_algorithm.png "DDPG algorithm")

## Hyper parameters (baseline)


| Parameter | Value | Comment |
| --- | --- | --- |
|BUFFER_SIZE |int(1e6)  | replay buffer size |
|BATCH_SIZE | 256        | minibatch size |
|GAMMA | 0.99            | discount factor |
|TAU | 1e-3              | for soft update of target parameters|
|LR_ACTOR | 1e-3         | learning rate of the actor |
|LR_CRITIC | 1e-3        | learning rate of the critic|
|WEIGHT_DECAY | 0        | L2 weight decay|
|EPSILON | 1.0           | epsilon for the noise process added to the actions|
|EPSILON_DECAY | 1e-6    | decay for epsilon above|
|NOISE_SIGMA | 0.05      | sigma for Ornstein-Uhlenbeck noise|
|UPDATE_EVERY | 20       | timesteps between updates|
|NUM_UPDATES | 10        | num of update passes when updating|
|Hidden layers| 1 | num of hidden layers in actor/critic networks|
|BatchNormalization | True | User batch normalization entering layers |
|Width hidden laysers | 128 | Number of neurons in the hidden layers |



## Results
Running with the hyper parameters above and using two layer 128 wide networks both for actor and critic networs (including batch normalization) I got the following training results reaching the goal of total average reward over the the last 100 episodes of 30 in 120 episodes. The problems was solved in 113 episodes. It was hard to make it better even after tuning and experimenting for quite some time. Some of the results of the experiments are described below.

![Resutls](training_results.png)

## Experiments
The below experiments were done with the base configuration above with the follwing changes. The goal reach is the number of iterations needed to reach the goal of an average score of 30 the last 100 iterations.

### Batch normalization
I added batch normalization between the layers with good effect on the speed of training (reaching goal in 133 vs. 199 epsisodes without batch normalization).

#### Yes: goal reach 113
Baseline
#### No:  goal reach 199
As expected the learning starts later, but the score is the similar.

### Hidden Network size (two layers)
#### 64:  goal reach 114
Still learning well.
#### 128:  goal reach 113
Baseline.
#### 256: goal reach 121
This was both slower in time and interations compared to the 128 variant. Too big for the problem I guess.

### Batch size
Batching is needed to be able to train the networks with a limited memory and computaion available. Bigger batch size is better, but not very much to gain going from 256 to 1024. The 256 batch size seems to contain enough variety in actions and states to train the networks.

#### 1024:  goal reach 112
#### 256:  goal reach 113
Baseline


### Buffer size
The replay buffer is essential in DDPG (and DPG) but having it bigger than 100 000 in this problem semms not to be necessary.
#### 100 000:  goal reach 110
Baseline.
#### 1 000 000:  goal reach 120
Maybe the oldest samples are out of date (i.e. based on an outdated policy) and that has some negative effect.

### Number of updates
This is the number of times the update of the target networks are done. It has some effect on the learning, but not much. 
#### 5:  goal reach 113
#### 10:  goal reach 113
Baseline
#### 15:   goal reach 115


## Future improvements

### There are ways to improve this even further by parallelizing the training using:
 * A2C
 * A3C
 * PPO
 
### Prioritized experience replay
Don't pick samples randomly, but prioritize the samples that give the most error which will give the most learnings for the networks.

### Other
Next time I make an agent I will make it easier to do grid-search for the best parameters, but spending all this time experimenting helped me understand the algorithm better - it is really impressive what it can do.

## References
I looked at several solutions to this problem but was most inspired by https://github.com/tommytracey/DeepRL-P2-Continuous-Control
